In [1]:
#import the necessary libaries
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
# nltk.download('stopwords') #run once and comment it out to avoid it downloading multiple times
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import string
import re
import textblob
from textblob import TextBlob

from wordcloud import WordCloud, STOPWORDS
from emot.emo_unicode import UNICODE_EMOJI

porter = PorterStemmer()

lemmatizer = WordNetLemmatizer() 


import warnings
%matplotlib inline

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

from collections import defaultdict
from textblob import TextBlob

In [2]:
#load the dataset
df_covid = pd.read_csv('covid19_tweets.csv')

In [3]:
#randomly select 1000 samples from the dataset
df = df_covid.sample(1000, random_state = 2)

In [4]:
df

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
53692,Aqib Shah,Srinagar I New Delhi,|Construction Liaison| Accredited Contractor ...,2009-08-22 15:19:33,75,187,312,False,2020-08-01 18:22:33,It ain't a rumour.We all know what as a joint ...,['COVID19'],Twitter for Android,False
82575,Black N White Tales,"Mumbai, India",Media Publishing & Entertainment 📽,2018-03-01 09:37:49,68,1345,8321,False,2020-08-07 12:55:08,one of my colleague... an editor just turned p...,['COVID19'],Twitter for Android,False
124373,RAWDEEP,South Africa,-Dj - Entrepreneur - Politically eclectic - pl...,2016-02-04 00:37:29,589,28,3985,False,2020-08-13 07:11:03,#COVID19 take off your mask and breathe deep,['COVID19'],Twitter for iPhone,False
176350,Prof. Gary Francione,New Jersey,"Professor, Rutgers University & University of ...",2009-05-31 08:01:42,11240,190,400,False,2020-08-29 22:23:15,NYC kill lists are suspended due to #COVID19. ...,['COVID19'],Twitter for iPhone,False
138855,Open Letters,Text RESIST to 50409,Copies of real letters delivered to the Presid...,2017-12-06 22:18:07,17341,3,2,True,2020-08-16 03:13:49,🖋 Sign “Authorize COVID-19 Rapid Testing” and ...,NaN,Resistbot Open Letters,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44704,Operator 12 - Digital Soldier,HEART OF THE BATTLE,"We are in this together! camo up patriots, th...",2020-03-21 04:00:56,361,410,1962,False,2020-07-31 19:15:03,Wake up. Your hero #Fauci just told all of you...,['Fauci'],Twitter for iPhone,False
143342,Outlook Magazine,"New Delhi, India",The fully loaded news magazine from India,2009-03-04 12:45:00,223384,57,221,True,2020-08-17 06:29:14,"Samaresh Das, three-time MLA from Egra assembl...",NaN,Twitter for Android,False
48983,BCMHouston,"Houston, Texas",Baylor College of Medicine is a health science...,2009-04-22 15:19:24,21262,1570,15923,True,2020-07-31 18:00:09,Today we launched our Resilience Through COVID...,NaN,Sprout Social,False
174628,Le-Basit,NaN,yes,2020-05-31 17:44:35,7,16,22,False,2020-08-30 00:32:22,Idiots ignoring all the proof \n#COVIDIOTS #CO...,"['COVIDIOTS', 'COVID19']",Twitter for Android,False


In [5]:
#checking for null values
df.isnull().sum()

user_name             0
user_location       166
user_description     51
user_created          0
user_followers        0
user_friends          0
user_favourites       0
user_verified         0
date                  0
text                  0
hashtags            284
source                0
is_retweet            0
dtype: int64

In [6]:
df = df.dropna()

In [7]:
df.shape

(589, 13)

In [8]:
#reset index of dataframe
df= df.reset_index()
df.index = df.index + 1
df = df.drop("index", axis = 1)  

In [9]:
df.head(5)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
1,Aqib Shah,Srinagar I New Delhi,|Construction Liaison| Accredited Contractor ...,2009-08-22 15:19:33,75,187,312,False,2020-08-01 18:22:33,It ain't a rumour.We all know what as a joint ...,['COVID19'],Twitter for Android,False
2,Black N White Tales,"Mumbai, India",Media Publishing & Entertainment 📽,2018-03-01 09:37:49,68,1345,8321,False,2020-08-07 12:55:08,one of my colleague... an editor just turned p...,['COVID19'],Twitter for Android,False
3,RAWDEEP,South Africa,-Dj - Entrepreneur - Politically eclectic - pl...,2016-02-04 00:37:29,589,28,3985,False,2020-08-13 07:11:03,#COVID19 take off your mask and breathe deep,['COVID19'],Twitter for iPhone,False
4,Prof. Gary Francione,New Jersey,"Professor, Rutgers University & University of ...",2009-05-31 08:01:42,11240,190,400,False,2020-08-29 22:23:15,NYC kill lists are suspended due to #COVID19. ...,['COVID19'],Twitter for iPhone,False
5,Alain Chivilò,Everywhere,Here only a quick view on different topics: la...,2009-07-19 15:32:03,3487,1379,96,False,2020-08-18 16:42:59,Global. \n+ Cases/Deaths To 23.000.000/ 800.00...,"['COVID19Updates', 'COVID19Update', 'CoronaVir...",Twitter for iPad,False


### Task 2.1 Data Preprocessing
The data is prepared for the sentiment analysis to be carried out on the tweets so characters, emojis, words that are not needed are filtered out in this process

In [10]:
eng_stop_words = list(stopwords.words('english'))

In [11]:
emoji = list(UNICODE_EMOJI.keys())

In [12]:
# function for preprocessing tweet in preparation for sentiment analysis
def ProcessedTweets(text):
    #changing tweet text to small letters
    text = text.lower()
    # Removing @ and links 
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", text).split())
    # removing repeating characters
    text = re.sub(r'\@\w+|\#\w+|\d+', '', text)
    # removing punctuation and numbers
    punct = str.maketrans('', '', string.punctuation+string.digits)
    text = text.translate(punct)
    # tokenizing words and removing stop words from the tweet text
    tokens = word_tokenize(text)  
    filtered_words = [w for w in tokens if w not in eng_stop_words]
    filtered_words = [w for w in filtered_words if w not in emoji]
    # lemmetizing words
    lemmatizer = WordNetLemmatizer() 
    lemma_words = [lemmatizer.lemmatize(w) for w in filtered_words]
    text = " ".join(lemma_words)
    return text

In [13]:
# Generate a new column called 'Processed Tweets' by applying preprocessed tweets function to the 'Tweet' column.
df['Processed_Tweets'] = df['text'].apply(ProcessedTweets)

In [14]:
df.tail(5)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,Processed_Tweets
585,Ryan Mayer,"Sacramento, CA","Catholic husband, father to 4, teacher. Bioeth...",2013-06-08 19:50:11,198,580,6700,False,2020-08-02 20:45:30,So a number of @MLB players have tested positi...,['COVID19'],Twitter Web App,False,number player tested positive sick dead asympt...
586,Rahul Gupta,India,Inbound Marketing Specialist | SEO Strategist ...,2013-11-13 06:27:39,36,223,142,False,2020-08-11 09:34:40,Finally the game has begun.\n\nRussia has made...,"['COVID19', 'oxfordvaccine']",Twitter for Android,False,finally game begun russia made world first vac...
587,Brian Chandler,Utah,Business and technology development executive ...,2012-08-25 05:23:57,447,383,12,False,2020-07-31 18:20:14,These places will pay remote workers to move t...,['COVID19'],Hootsuite Inc.,False,place pay remote worker move boost covid recovery
588,Suzanne Schmidt,"Los Angeles, CA","Fueled by #THEATRE, #LAUGHTER, & #HUMANITY. #s...",2012-07-11 17:31:08,300,499,2220,False,2020-08-14 02:01:14,If we tell @POTUS that #COVID19 is not a deadl...,"['COVID19', 'Leftist', 'BIPOC', 'immigrant', '...",Twitter for iPhone,False,tell deadly virus rather group amp
589,Operator 12 - Digital Soldier,HEART OF THE BATTLE,"We are in this together! camo up patriots, th...",2020-03-21 04:00:56,361,410,1962,False,2020-07-31 19:15:03,Wake up. Your hero #Fauci just told all of you...,['Fauci'],Twitter for iPhone,False,wake hero told trump saved life maybe medium l...


In [15]:
#number of uniue location
df['user_location'].nunique()

427

In [16]:
df.shape

(589, 14)

In [17]:
#the unique locations in the dataframe
df['user_location'].unique()

array(['Srinagar I New Delhi', 'Mumbai, India', 'South Africa ',
       'New Jersey', 'Everywhere', 'Harpenden', 'India', 'Pakistan',
       'Manila, Philippines', 'US', 'Vancouver, BC', 'Mumbai',
       'BHUBANESWAR', 'Boulder, CO', 'Toronto, Ontario',
       'Campbell, CA ⇄ Maui, HI', 'Florida, USA', 'Orange County, CA',
       'Alberta & Treaty 6', 'Washington, D.C.', 'Chicago Native 🌍🇬🇲',
       'New Delhi-110029', 'Lahore, Pakistan', 'Düsseldorf, Germany',
       'Noida', 'Dungun, Terengganu', 'Thessaloniki, Greece',
       'south wales', 'Somerset, UK', 'Everywhere I Want To Be....',
       'Los Angeles, CA', 'Chennai, India',
       'Base of Pikes Peak, Colorado', 'Paterson, NJ', 'Morrison CO',
       '#Lagos ◄ ► #CoalCity', 'Singapore', 'Auckland Park',
       'Central Florida', 'Thorpe-le-Soken, England', 'Australia',
       'New Zealand', 'Winter Park, FL; MetroWest, FL',
       'Srinagar, Kashmir', 'Colorado Springs, CO', 'Bombay',
       'Colombo, SriLanka', 'UK', 'United K

In [18]:
#locations in the dataframe which are not actual locations are removed
to_remove = ['Everywhere I Want To Be....', '#Lagos ◄ ► #CoalCity', 'Between deadlines and humans', 'world','11-4- Chilkalguda secunderabad',
            'UK | USA | UAE | CAN | CHN | NL','Earth','Yuin people Walbunja Clan','Wherever threads are written..', 'SickSadWorld',
            'East of West', 'Paris75004','Hogwarts, TERF Island','World Wide','  NO investment advice given', 'United Kingdom, India  she/her',
            'In your head!', 'Not a State of Hate! ','Wherever I hang my hat', '#Noise #Community #QoL #MeToo ', '21°45′N 71°35′W',
            'JoeKamalaLand! 🏳️\u200d🌈🇺🇸 🆘','World Citizen', 'Gurgaon/New York/Bay Area (CA)', 'A little cave near the beach..',
            'my own state of PEACE of mind','☀ - ♌ || 🌔 - ♋ || Asc - ♍ ','Global','My kitchen',
            'World Wide Web, & Colorado ','#AFRICA #MENA', '@CapricornFMNews','ÜT: 51.480992,-3.194113',
            'A fridge in Moscow', 'HEART OF THE BATTLE', 'Avi is me','no e-pass to cross borders...','None Of Your Damned Business!!',
            'Palm Springs / Los Angeles / Houston / Miami',
            '51.813984,-0.025493', 'l London I New York I','Adana & Tbilisi ','Everywhere',
            'WORLDWIDE','MD, NoVA, DC, So PA','The best Country on Earth.','hyderabad','Japan or a fairy circle','.','Over 100 countries worldwide',
            'Grave','Bogota/Philadelphia/Lewisburg','worldwide',' Mysterious Midwest','Geneva and Athens',]
mask = df['user_location'].isin(to_remove)
df = df[~mask]

In [19]:
#function to rename some locations
def standardize_location(location):
    location = location.replace('🇨🇦', 'California')
    location = location.replace('Base of Pikes Peak, Colorado', 'California')
    location = location.replace('Campbell, CA ⇄ Maui, HI', 'Campbell, California')
    location = location.replace('BHUBANESWAR', 'Bhubaneswar, India')
    location = location.replace('Mumbai, India india', 'Mumbai, India')
    location = location.replace('Chicago Native 🌍🇬🇲', 'Native American, Chicago')
    location = location.replace('God Bless the U.S.A.!!', 'United States of America')
    location = location.replace('Suomi | Finland 🇫🇮', 'Finland')
    location = location.replace('Jammu, Jammu And Kashmir', 'Jammu And Kashmir, India')
    location = location.replace('Alberta & Treaty 6', 'Alberta')
    return location

In [20]:
df['user_location'] = df['user_location'].apply(standardize_location)

## Task 2.2 Geocoding

In [21]:
#calling the Geocoding API
locator = Nominatim(user_agent="myGeocoder")

In [22]:
# Import the RateLimiter class from the geopy.extra.rate_limiter module
from geopy.extra.rate_limiter import RateLimiter

# Create a RateLimiter object that wraps the locator.geocode function, with a minimum delay of 1 second between requests
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

# Iterate over each row in the DataFrame df
for index, row in df.iterrows():
    try:
        # Call the geocode function with the 'user_location' value in the current row
        # The geocode function returns a Location object that contains the latitude, longitude, and altitude coordinates of the location
        # The RateLimiter ensures that the geocode function is called at a limited rate to avoid overloading the geocoding service
        location = geocode(row['user_location'])
        # Convert the location object to a tuple of (latitude, longitude, altitude) coordinates
        # If location is None, set point to None as well
        point = tuple(location.point) if location else None
    except Exception as e:
        # If an exception occurs while calling the geocode function, set point to None and print an error message

        point = None
        print(f"Error processing row {index}: {e}")
        # Set the 'latitude', 'longitude', and 'altitude' values in the DataFrame for the current row
    # If point is None, set the values to None as well
    
    if point:
        df.at[index, 'latitude'] = point[0]
        df.at[index, 'longitude'] = point[1]
        df.at[index, 'altitude'] = point[2]
    else:
        df.at[index, 'latitude'] = None
        df.at[index, 'longitude'] = None
        df.at[index, 'altitude'] = None


In [23]:
df.head(5)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,Processed_Tweets,latitude,longitude,altitude
1,Aqib Shah,Srinagar I New Delhi,|Construction Liaison| Accredited Contractor ...,2009-08-22 15:19:33,75,187,312,False,2020-08-01 18:22:33,It ain't a rumour.We all know what as a joint ...,['COVID19'],Twitter for Android,False,aint rumourwe know joint commissioner fight ha...,34.084423,74.822682,0.0
2,Black N White Tales,"Mumbai, India",Media Publishing & Entertainment 📽,2018-03-01 09:37:49,68,1345,8321,False,2020-08-07 12:55:08,one of my colleague... an editor just turned p...,['COVID19'],Twitter for Android,False,one colleague editor turned positive admitted ...,19.078545,72.878176,0.0
3,RAWDEEP,South Africa,-Dj - Entrepreneur - Politically eclectic - pl...,2016-02-04 00:37:29,589,28,3985,False,2020-08-13 07:11:03,#COVID19 take off your mask and breathe deep,['COVID19'],Twitter for iPhone,False,take mask breathe deep,-28.816624,24.991639,0.0
4,Prof. Gary Francione,New Jersey,"Professor, Rutgers University & University of ...",2009-05-31 08:01:42,11240,190,400,False,2020-08-29 22:23:15,NYC kill lists are suspended due to #COVID19. ...,['COVID19'],Twitter for iPhone,False,nyc kill list suspended due euthanasia unless ...,40.075738,-74.404162,0.0
6,Asset TV,Harpenden,"The secure video platform for due diligence, v...",2009-09-14 14:10:13,4330,2265,748,True,2020-08-01 18:50:00,How has #COVID19 affected #cybercrime? Find ou...,"['COVID19', 'cybercrime']",Twitter Media Studio,False,affected find akademia short john harrison hea...,51.814510,-0.354931,0.0


In [24]:
#to indicate which value is null
mask = df.isnull().any(axis=1)

In [25]:
df_null = df[mask]
df_null

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,Processed_Tweets,latitude,longitude,altitude
21,📚,"Native American, Chicago",Be You.UW-Madison Alum.💕♻️♒️🙏🏿 #HealthIsWealth...,2017-05-29 05:00:17,234,1289,40764,False,2020-07-26 04:06:17,Herd Immunity explained @SenKamalaHarris @Kama...,"['CoronaVirusUpdates', 'coronavirus', 'coronav...",Twitter for iPhone,False,herd immunity explained,NaN,NaN,NaN
45,Central Florida Primary Care,"Winter Park, FL; MetroWest, FL",Healthcare with Quality is our #1 priority.,2019-12-10 18:40:49,32,165,14,False,2020-08-06 15:07:39,"There are now 510,389 + confirmed cases of #CO...",['COVID19'],Twitter Web App,False,confirmed case across sunshine state per fldoh,NaN,NaN,NaN
58,ircwash,"Africa, Asia and Latin America",International think and do tank supporting wat...,2008-08-28 09:13:03,20809,1998,2764,False,2020-08-13 07:53:19,".@circleofblue speaks to Vida Duti, IRC Ghana ...","['COVID19', 'Ghana']",Twitter for Advertisers,False,speaks vida duti irc ghana country director le...,NaN,NaN,NaN
79,Private Suite Network | John Cutler,NJ NYC Philly AC Central NJ,Tickets | Luxury Suites | Single Event Rentals...,2011-01-07 12:51:35,3034,4535,31350,False,2020-08-01 18:33:46,"N.J. reports 11 new coronavirus deaths, 393 ne...","['NJ', 'NewJersey']",Twitter for Android,False,nj report new coronavirus death new case rate ...,NaN,NaN,NaN
96,Dayo Ntwari 🖊️🎙️,Chuckletown,"Father, Husband, #Writer🖊️ #Podcaster🎙️ @Logfi...",2015-01-02 04:35:20,720,790,30147,False,2020-08-30 08:52:20,@Chungsinrwanda @nsanzimanasabin @MoHRwanda @i...,['COVID19'],Twitter for Android,False,there herd immunity dangerous n,NaN,NaN,NaN
102,MANIK SING,Kadabahali,ଜନ୍ମ ତାରିଖ ୦୮/୦୫/୧୯୯୮\nସ୍ଥାନ ବଡ଼ କାଦବାହାଳୀ,2018-05-09 20:31:06,1,53,83,False,2020-07-27 07:06:12,New video by OTV: #COVID19 BREAKING-864 More R...,['COVID19'],IFTTT,False,new video otv breaking recuperate disease odisha,NaN,NaN,NaN
127,💵🦈 Shark_Stocks 🦈💵,Deep under Water,Risk vs. Reward.... Too much fun right?🚨 #shar...,2017-10-20 01:36:55,2472,384,7393,False,2020-08-16 02:53:48,$TPTW. Listen to first 2 min here @realDonaldT...,"['Walmart', 'CEO', 'COVID19']",Twitter for iPhone,False,tptw listen first min bring talk testing par,NaN,NaN,NaN
150,Penelope Silvers,Florida Gulf of Mexico,Visit 🔜https://t.co/BQHGjGDfm7 for 3 Free Chap...,2012-07-18 17:03:48,5631,6323,20689,False,2020-08-18 16:14:00,If Congress can't reach an agreement by Friday...,['Stimuluscheck'],Twitter Web App,False,congress cant reach agreement friday american ...,NaN,NaN,NaN
152,Legal Affairs Canada,Guelph Toronto Canada Scotland,Focused on Supreme Court Canada plus law right...,2011-09-25 18:07:11,6437,1681,13441,False,2020-08-18 18:07:15,There is extensive material specifically for c...,['covid19'],Twitter Web App,False,extensive material specifically court small co...,NaN,NaN,NaN
167,Zhao Yanbo,Plot 3096 North Ring Road,Chinese Ambassador to Botswana,2020-02-27 13:11:41,1837,1023,1956,False,2020-08-12 08:08:12,Yesterday President #XiJinping signed a presid...,['XiJinping'],Twitter Web App,False,yesterday president signed presidential order ...,NaN,NaN,NaN


In [26]:
df.shape

(527, 17)

In [27]:
df.isnull().sum()

user_name            0
user_location        0
user_description     0
user_created         0
user_followers       0
user_friends         0
user_favourites      0
user_verified        0
date                 0
text                 0
hashtags             0
source               0
is_retweet           0
Processed_Tweets     0
latitude            27
longitude           27
altitude            27
dtype: int64

In [28]:
df = df.dropna()

In [29]:
df= df.reset_index()
df.index = df.index + 1
df = df.drop("index", axis = 1)

In [30]:
df.head(30)

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,Processed_Tweets,latitude,longitude,altitude
1,Aqib Shah,Srinagar I New Delhi,|Construction Liaison| Accredited Contractor ...,2009-08-22 15:19:33,75,187,312,False,2020-08-01 18:22:33,It ain't a rumour.We all know what as a joint ...,['COVID19'],Twitter for Android,False,aint rumourwe know joint commissioner fight ha...,34.084423,74.822682,0.0
2,Black N White Tales,"Mumbai, India",Media Publishing & Entertainment 📽,2018-03-01 09:37:49,68,1345,8321,False,2020-08-07 12:55:08,one of my colleague... an editor just turned p...,['COVID19'],Twitter for Android,False,one colleague editor turned positive admitted ...,19.078545,72.878176,0.0
3,RAWDEEP,South Africa,-Dj - Entrepreneur - Politically eclectic - pl...,2016-02-04 00:37:29,589,28,3985,False,2020-08-13 07:11:03,#COVID19 take off your mask and breathe deep,['COVID19'],Twitter for iPhone,False,take mask breathe deep,-28.816624,24.991639,0.0
4,Prof. Gary Francione,New Jersey,"Professor, Rutgers University & University of ...",2009-05-31 08:01:42,11240,190,400,False,2020-08-29 22:23:15,NYC kill lists are suspended due to #COVID19. ...,['COVID19'],Twitter for iPhone,False,nyc kill list suspended due euthanasia unless ...,40.075738,-74.404162,0.0
5,Asset TV,Harpenden,"The secure video platform for due diligence, v...",2009-09-14 14:10:13,4330,2265,748,True,2020-08-01 18:50:00,How has #COVID19 affected #cybercrime? Find ou...,"['COVID19', 'cybercrime']",Twitter Media Studio,False,affected find akademia short john harrison hea...,51.814510,-0.354931,0.0
6,NDTV,India,Breaking news alerts from India.\n\nInstagram:...,2009-05-01 20:34:48,13477333,15,1,True,2020-08-11 09:04:03,#JustIn | Madhya Pradesh CM Shivraj Singh Chou...,"['JustIn', 'COVID19']",Twitter Web App,False,madhya pradesh cm shivraj singh chouhan tweet ...,22.351115,78.667743,0.0
7,Pakistan Professional Women Forum,Pakistan,Pakistan Professional Women Forum gives a plat...,2019-03-20 17:13:32,587,51,99,False,2020-07-25 09:39:17,"According to their findings, #remotework and s...","['remotework', 'virtualmeetings', 'genderinequ...",Twitter for Android,False,according finding specifically may exacerbate,30.330840,71.247499,0.0
8,ABS-CBN News,"Manila, Philippines","Stories, video, and multimedia for Filipinos w...",2008-08-16 10:09:33,7040042,1070,1159,True,2020-08-14 05:35:00,PH in talks with at least 16 #COVID19 vaccine ...,['COVID19'],TweetDeck,False,ph talk least vaccine developer doh,14.594891,120.978262,0.0
9,Doozie,US,I have an opinion. Just ask me.,2011-11-03 00:54:09,46,255,3089,False,2020-08-02 21:34:46,@TomFitton Any media source or pharmacy preven...,['covid19'],Twitter for iPhone,False,medium source pharmacy preventing med getting ...,39.783730,-100.445882,0.0
10,Canada's Fight Against COVID-19,"Vancouver, BC",Bot created for the purpose of sharing numbers...,2020-03-27 23:46:09,12,0,0,False,2020-08-02 20:00:26,"Currently, there are a total of 116,858 confir...","['Canada', 'COVID19']",Canada COVID-19 Bot,False,currently total confirmed case death,49.260872,-123.113952,0.0


In [31]:
df = df.dropna()

In [32]:
df.shape

(500, 17)

In [33]:
# df.to_csv('new_covid.csv')

## Sentiment Analysis
This is to get the sentiments of the tweets in the dataframe

In [34]:
# Function for polarity score
def polarity(tweet):
    return TextBlob(tweet).sentiment.polarity

# Function to get sentiment type
#setting the conditions
def sentimenttextblob(polarity):
    if polarity < 0:
        return "Negative"
    elif polarity == 0:
        return "Neutral"
    else:
        return "Positive" 

In [35]:
# using the functions to get the polarity and sentiment
df['Polarity'] = df['Processed_Tweets'].apply(polarity)
df['Sentiment'] = df['Polarity'].apply(sentimenttextblob)
sent = df['Sentiment'].value_counts()
sent

Neutral     231
Positive    189
Negative     80
Name: Sentiment, dtype: int64

In [36]:
# Create a dictionary to store the sum of polarity values and number of tweets for each location
location_polarity_sum = defaultdict(float)
location_tweet_count = defaultdict(int)

# Iterate over each tweet and add its polarity value to the sum for its location
for _, tweet in df.iterrows():
    blob = TextBlob(tweet["Processed_Tweets"])
    polarity = blob.sentiment.polarity
    location_polarity_sum[(tweet["latitude"], tweet["longitude"])] += polarity
    location_tweet_count[(tweet["latitude"], tweet["longitude"])] += 1

# Create a new dataframe to store the average polarity values for each location
locations_df = pd.DataFrame(columns=["latitude", "longitude", "polarity"])

# Iterate over each location and calculate the average polarity value
for location, polarity_sum in location_polarity_sum.items():
    latitude, longitude = location
    tweet_count = location_tweet_count[location]
    average_polarity = polarity_sum / tweet_count
    locations_df = locations_df.append({"latitude": latitude, "longitude": longitude, "polarity": average_polarity}, ignore_index=True)


C:\Users\Flora\AppData\Local\Temp\ipykernel_15140\776588459.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  locations_df = locations_df.append({"latitude": latitude, "longitude": longitude, "polarity": average_polarity}, ignore_index=True)
C:\Users\Flora\AppData\Local\Temp\ipykernel_15140\776588459.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  locations_df = locations_df.append({"latitude": latitude, "longitude": longitude, "polarity": average_polarity}, ignore_index=True)
C:\Users\Flora\AppData\Local\Temp\ipykernel_15140\776588459.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  locations_df = locations_df.append({"latitude": latitude, "longitude": longitude, "polarity": average_polarity}, ignore_index=True

In [37]:
locations_df

,latitude,longitude,polarity
0,34.084423,74.822682,0.000000
1,19.078545,72.878176,0.047619
2,-28.816624,24.991639,0.170000
3,40.075738,-74.404162,-0.062500
4,51.814510,-0.354931,0.000000
...,...,...,...
280,38.475841,-80.840841,0.136364
281,38.725178,-105.607716,-0.100000
282,35.134513,33.922586,0.000000
283,38.581061,-121.493895,-0.229004


**Geospatial Analysis of the Tweets Sentiments**

## Task 2.3 Polarity Analysis

In [38]:
# Create a map centered at the mean coordinates of all the tweets
map = folium.Map(location=[locations_df["latitude"].mean(), locations_df["longitude"].mean()], zoom_start=10)

# Iterate over each tweet and add a marker to the map
for index, row in locations_df.iterrows():
    # Get the polarity value of the tweet
    polarity = row["polarity"]
    
    # Set the color of the marker based on the polarity value
    if polarity > 0:
        color = "green"
    elif polarity < 0:
        color = "red"
    else:
        color = "gray"
    
    # Add a marker to the map
    folium.Marker(location=[row["latitude"], row["longitude"]], icon=folium.Icon(color=color)).add_to(map)

# Display the map
map


##  Task 2.4 Subjectivity Analysis

In [39]:
# calculate the subjectivity values of all the tweets
df['Subjectivity'] = df['Processed_Tweets'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

# group the tweets by their geographical location and calculate the average subjectivity value
grouped_tweets_df = df.groupby(['longitude', 'latitude']).agg({'Subjectivity': 'mean'}).reset_index()

# create a map centered at the mean location of all the tweets
mean_lat = df['latitude'].mean()
mean_long = df['longitude'].mean()
tweets_map = folium.Map(location=[mean_lat, mean_long], zoom_start=2)

# add a scatter plot to the map
for index, row in grouped_tweets_df.iterrows():
    lat = row['latitude']
    long = row['longitude']
    subjectivity = row['Subjectivity']
    # set the color based on the subjectivity value
    if subjectivity > 0.5:
        color = 'green'
    elif subjectivity < 0.5:
        color = 'red'
    else:
        color = 'gray'
    folium.CircleMarker(location=[lat, long], radius=5, color=color, fill=True, fill_color=color).add_to(tweets_map)

# show the map
tweets_map


In [40]:
# Function for polarity score
def subject(tweet):
    return TextBlob(tweet).sentiment.subjectivity

def subjectivityttextblob(subjectivity):
    if subjectivity > 0.5:
        return "Subjective"
    elif subjectivity < 0.5:
        return 'Objective'
    else:
        return "Neutral"

In [41]:
df['Subjective_category'] = df['Subjectivity'].apply(subjectivityttextblob)

In [42]:
sub = df['Subjective_category'].value_counts()
sub

Objective     344
Subjective    126
Neutral        30
Name: Subjective_category, dtype: int64

## Task 2.5 Storify

#### Opinion of people regarding Covid-19
As a policy advisor to the UK government giving an update on about the public sentiment related to Covid-19 and its vaccine rollout. From the result of the sentiment analysis it shows that 

- People across the world has sentiments spanning across **Neutral, Positive and Negative** sentiments in the ratio 3:2:1 concerning Covid-19. The public opinion also were **Objective, Subjective and Neutral** in a ratio 11:4:1

- From the map North America had a more negative sentiment about Covid-19 while Europe had a more Positive Sentiment about it. Africa also has mixed sentiment between positive and neutral. The opinion generally about covid-19 across all continent was mostly Objective there was just little subjectivity on the opinion given in North America, Europe and Australia while the neutrality was minimal.

- Since UK is the country most likely to derive a government policy from the tweets of its citizen then its best to focus on the tweets gotten from that region. So in UK it was seen that these locations **Dublin, Northern Ireland, Carlisle, New Castle Upon Tyne, Northallerton, Winchester and Nottingham** has a negative sentiment towards Covid-19 which implies that people from those region has a negative view about the Covid-19 outbreak. From a tweet taken from the UK which has a negative sentimen and also subjective which goes thus "Why COVID-19 is killing diabetes patients at alarming rates" which is not followed by a verified source this tweet most likely is hinged on spreading false rumor and fear to the public about covid-19. Unlike a more positive inclined tweet like this "How many of us became content creators during the #COVID19 lockdown?" 

- Tweets from locations that are negative and more inclined to be subjective should be looked into more as they could be a way of spreading lies based on emotional constraint attached to being in a pandemic. Since tweets like that are most likely to go viral then there should be sensitization on the spread of false news over tweets.